In [45]:
import pandas as pd
import numpy as np

import string  
import nltk
import re
#import scipy
#import matplotlib.pyplot as plt
#import urllib 
import sklearn
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
display(train)
display(test)  

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [46]:
#limpiando el texto

import nltk
# nltk.download('stopwords') 
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

def limpiar_tweets():
    #Cleaning text
    train['clean_text'] = train['text'].str.lower()
    train['clean_text'] = train['clean_text'].apply(only_letters)    
    test['clean_text'] = test['text'].str.lower()
    test['clean_text'] = test['clean_text'].apply(only_letters) 
    #Tokenización
    train['clean_text'] = train['clean_text'].apply(word_tokenize)
    test['clean_text'] = test['clean_text'].apply(word_tokenize)
    #Remove stopwords
    train['clean_text'] = train['clean_text'].apply(filter_stopwords) 
    test['clean_text'] = test['clean_text'].apply(filter_stopwords) 
    #Lemmatization                                                                       
    train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  
    train['clean_text'] = train['clean_text'].apply(transform_to_text)
    test['clean_text'] = test['clean_text'].apply(lemmatize_tweet)  
    test['clean_text'] = test['clean_text'].apply(transform_to_text)

#display(train)

**TF-IDF pruebo sobre los tweets "limpios" sin y con función de tokenización**

In [99]:
tf_idf = TfidfVectorizer(min_df = 10, max_df = 0.80, ngram_range= (1,2)) #sin función de tokenización

limpiar_tweets()
#Corremos el algoritmo para TRAIN set 
texts = train['clean_text']
features = tf_idf.fit_transform(texts)
feature_words = tf_idf.get_feature_names()
df_tf_idf = pd.DataFrame(data = features.todense(), columns = tf_idf.get_feature_names())
df_tf_idf["target"] = train.target

In [100]:
#Corremos el algoritmo para TEST set 
texts_test = test['clean_text']

features_test = tf_idf.transform(texts_test)

feature_words_test = tf_idf.get_feature_names()

df_tf_idf_test = pd.DataFrame(data = features_test.todense(), columns = tf_idf.get_feature_names())
df_tf_idf_test.head()

,aba,aba woman,abandoned,abc,abc news,ablaze,able,absolutely,accident,according,...,youth,youth saved,youtube,youtube playlist,youtube video,youve,yr,yr old,yyc,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
#Construimos los datos de entrenamiento y de test
y = df_tf_idf["target"].values
X = df_tf_idf.drop(["target"], axis=1).values

X_TEST = df_tf_idf_test

X_train_tf_idf, X_test_tf_idf, y_train_tf_idf, y_test_tf_idf = train_test_split(X, y, test_size=0.20, random_state=41)

In [102]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf_idf, y_train_tf_idf)
preds = naive_bayes.predict(X_test_tf_idf)

print('f1_score: ', f1_score(y_test_tf_idf, preds))
print('Accuracy: ', accuracy_score(y_test_tf_idf, preds))
print('Recall: ', recall_score(y_test_tf_idf, preds))
print('Precision: ', precision_score(y_test_tf_idf, preds))


f1_score:  0.762384550797649
Accuracy:  0.814182534471438
Recall:  0.6920731707317073
Precision:  0.8485981308411215


In [103]:
preds_train = naive_bayes.predict(X_train_tf_idf)
print('f1_score for train: ', f1_score(y_train_tf_idf, preds_train))
print('Accuracy for train: ', accuracy_score(y_train_tf_idf, preds_train))

f1_score for train:  0.7725431677680664
Accuracy for train:  0.8247947454844007


In [104]:
tf_idf = TfidfVectorizer(min_df = 10, max_df = 0.80, ngram_range= (1,2), tokenizer = nltk.word_tokenize)
#Denuevo ahora con función de tokenización

#Corremos el algoritmo para TRAIN set 
texts = train['clean_text']
features = tf_idf.fit_transform(texts)
feature_words = tf_idf.get_feature_names()
df_tf_idf = pd.DataFrame(data = features.todense(), columns = tf_idf.get_feature_names())
df_tf_idf["target"] = train.target

In [105]:
#Corremos el algoritmo para TEST set 
texts_test = test['clean_text']

features_test = tf_idf.transform(texts_test)

feature_words_test = tf_idf.get_feature_names()

df_tf_idf_test = pd.DataFrame(data = features_test.todense(), columns = tf_idf.get_feature_names())

df_tf_idf_test.head()

,aba,aba woman,abandoned,abc,abc news,ablaze,able,absolutely,accident,according,...,youth saved,youtube,youtube playlist,youtube video,youve,yr,yr old,yyc,z,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
#Construimos los datos de entrenamiento y de test

y = df_tf_idf["target"].values
X = df_tf_idf.drop(["target"], axis=1).values

X_TEST = df_tf_idf_test

X_train_tf_idf, X_test_tf_idf, y_train_tf_idf, y_test_tf_idf = train_test_split(X, y, test_size=0.20, random_state=41)

In [107]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf_idf, y_train_tf_idf)
preds = naive_bayes.predict(X_test_tf_idf)

print('f1_score: ', f1_score(y_test_tf_idf, preds))
print('Accuracy: ', accuracy_score(y_test_tf_idf, preds))
print('Recall: ', recall_score(y_test_tf_idf, preds))
print('Precision: ', precision_score(y_test_tf_idf, preds))

f1_score:  0.7596638655462186
Accuracy:  0.8122127380170716
Recall:  0.6890243902439024
Precision:  0.846441947565543


In [108]:
preds_train = naive_bayes.predict(X_train_tf_idf)
print('f1_score for train: ', f1_score(y_train_tf_idf, preds_train))
print('Accuracy for train: ', accuracy_score(y_train_tf_idf, preds_train))

f1_score for train:  0.7713310580204779
Accuracy for train:  0.8239737274220033


In [ ]:
#No mejoró mucho con respecto al que no utiliza la función de tokenización, ni tampoco da ninguno de los dos un mejor resultado
#que utilizando count vectorizer